In [1]:
# NAO USAR A TABELA data_product, E SIM A store_product!!

# Remover primeiro os itens 'C/ 8', verificando se após o 'C/' a string comça com dígito.
# Após isso, remover diretamente os itens com 'C/', por exemplos 'C/8'
# Fazer o mesmo para os itens com a string 'S/' e 'REF'

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('inputs/data_product_202208091047.csv', usecols = ['id', 'barcode', 'name'])

In [4]:
df.shape

(69111, 3)

In [5]:
df['name'].nunique()


48879

In [6]:
df.drop_duplicates(subset = 'name', inplace = True)

In [7]:
df.shape

(48879, 3)

In [8]:
df['name'] = df['name'].astype(str)

In [9]:
df['name'] = df['name'].str.upper()

In [10]:
df['cleaned'] = df['name']

In [31]:
def remove_with_successor(name, element):
    name = name.split(' ')

    for k, v in enumerate(name):
        try:
            if (v == element and name[k + 1][0].isdigit()) or (v == element):
                name.remove(name[k + 1])
                name.remove(v)
        except:
            pass
                    
    return ' '.join(name)

In [13]:
pairs = ['REF', 'R', 'C/', 'S/']

In [14]:
for item in pairs:
    df['cleaned'] = df['cleaned'].apply(lambda x: remove_with_successor(x, item))

In [15]:
# def remove_metrics(name):
#     to_remove = []
    
#     for word in name.split():
#         if word[0].isdigit() and 'X' in word and word[-1] == 'M' :
#             to_remove.append(word)

#     if len(to_remove) > 0:
#         return ''. join([name.replace(x, '') for x in to_remove])
    
#     return name

In [16]:
# df['cleaned'] = df['cleaned'].apply(lambda x: remove_metrics(x))

In [17]:
# def remove_startswith(name):
#     return lambda words: words.replace(''.join([x for x in words.split(' ') if x.startswith(name)]), '')

In [18]:
# startswith_ = ['(', 'REF', 'C/', 'S/', 'INTRAV']

In [19]:
# def remove_endswith(name, numbers):
#     return lambda words: words.replace(''.join([x for x in words.split(' ') if x.endswith(name) and x[:numbers].isdigit()]), '')

In [20]:
# endswith_ = {
#     'G':    -len('G'),
#     'L':    -len('L'),
#     'UN':   -len('UN'),
#     'ML':   -len('ML'),
#     'GR':   -len('GR'),
#     'CPS':  -len('CPS'),
#     'VOL':  -len('VOL'),
#     'CAP':  -len('CAP')
# }

In [21]:
# for name in startswith_:
#     df['cleaned'] = df['cleaned'].apply(remove_startswith(name))

In [22]:
# for k, v in endswith_.items():
#     df['cleaned'] = df['cleaned'].apply(remove_endswith(k, v))

In [23]:
df['cleaned'] = df['cleaned'].str.strip()

In [24]:
df['num_words'] = df['cleaned'].apply(lambda x: len(str(x).split(' ')))

In [25]:
items_cleaned = [
    52, 84, 15533, 15876
    # 48, 57, 58, 60, 76, 77, 82, 83, 84, 88, 94, 102, 106, 116, 118, 122, 171, 174
]

In [26]:
df[df['id'].isin(items_cleaned)].sort_values(by = 'id')

,id,barcode,name,cleaned,num_words
51,52,7897371801274,CHUP NUK R 729532,CHUP NUK,2
83,84,7898312327488,ESC PROART BAMBU REF 73252,ESC PROART BAMBU,3
15546,15533,7894900530025,AGUA CRYSTAL PET 1L C/ BICO,AGUA CRYSTAL PET 1L,4
15892,15876,7891025102113,AGUA MINERAL BONAFONT S/ GAS 500ML,AGUA MINERAL BONAFONT 500ML,4


In [27]:
# df.sort_values(by = 'cleaned', inplace = True)

In [28]:
# df.to_csv('output.csv')

---

Tests:

---

To remove:

In [29]:
df[df['id'].isin([56, 63, 84, 169, 14875])]

,id,barcode,name,cleaned,num_words
55,56,7897371800505,CHUP NUK R729244,CHUP NUK R729244,3
62,63,7891962004433,"AMANTEIGADO BANANA BAUDUCCO 11,5G","AMANTEIGADO BANANA BAUDUCCO 11,5G",4
83,84,7898312327488,ESC PROART BAMBU REF 73252,ESC PROART BAMBU,3
167,169,7898225526923,ADA TINA INPELE CAPELLI FORTI 60 CPS 30G,ADA TINA INPELE CAPELLI FORTI 60 CPS 30G,8
14891,14875,7896902200913,ACETONA FARMAX C/ 100 ML,ACETONA FARMAX ML,3


In [30]:
# dict_ = {
#     'SAB': 'SABONETE',
#     'DES': 'DESODORANTE',
#     'ESC': 'ESCOVA',
#     'LOC': 'LOCAO',
#     'TRAD': 'TRADICIONAL',
#     'CURAT': 'CURATIVO',
# }